# CDP - 236370
# Tutorial 4 - Numba jit & cuda

Written by Ido Hakimi, Qasem Sayah, and Anny Firer.
<br>
Adapted from https://nyu-cds.github.io/python-numba/ & https://numba.pydata.org/numba-doc/dev/user/5minguide.html

In [ ]:
!pip install --upgrade pip
!pip install numba

Decorators are a way to uniformly modify functions in a particular way. You can think of them as functions that take functions as input and produce a function as output. See the Python reference documentation for a detailed discussion (https://docs.python.org/3/reference/compound_stmts.html#function-definitions).

A function definition may be wrapped by one or more decorator expressions. Decorator expressions are evaluated when the function is defined, in the scope that contains the function definition. The result must be a callable, which is invoked with the function object as the only argument. The returned value is bound to the function name instead of the function object. Multiple decorators are applied in nested fashion. For example, the following code:

In [ ]:
def my_decorator(func):
    def wrapper(string):
        print("Something is happening before the function is called.")
        func(string)
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee(string):
    print(string)

In [ ]:
say_whee("Whee!")

## Just-In-Time (JIT)

JIT is a way of executing computer code that involves compilation during execution of a program – at run time – rather than prior to execution. Most often, this consists of source code or more commonly bytecode translation to machine code, which is then executed directly. A system implementing a JIT compiler typically continuously analyses the code being executed and identifies parts of the code where the speedup gained from compilation or recompilation would outweigh the overhead of compiling that code. 

JIT compilation is a combination of the two traditional approaches to translation to machine code – ahead-of-time compilation (AOT), and interpretation – and combines some advantages and drawbacks of both.


In [ ]:
from numba import jit

@jit
def f(x, y):
    # A somewhat trivial example
    return x + y

## Explicit Parallel Loops

Another feature of this code transformation pass is support for explicit parallel loops. One can use Numba’s prange instead of range to specify that a loop can be parallelized. The user is required to make sure that the loop does not have cross iteration dependencies except for supported reductions.

A reduction is inferred automatically if a variable is updated by a binary function/operator using its previous value in the loop body. The initial value of the reduction is inferred automatically for += and *= operators. For other functions/operators, the reduction variable should hold the identity value right before entering the prange loop. Reductions in this manner are supported for scalars and for arrays of arbitrary dimensions.

The example below demonstrates a parallel loop with a reduction (A is a one-dimensional Numpy array):

In [ ]:
from numba import njit, prange
@njit(parallel=True)
def prange_test(A):
    s = 0
    for i in prange(A.shape[0]):
        s += A[i]
    return s

## Numba Compilation

Numba’s central feature is the `numba.jit()` decoration. Using this decorator, it is possible to mark a function for optimization by Numba’s JIT compiler. Various invocation modes trigger differing compilation options and behaviours.

Numba has two compilation modes: *nopython mode* and *object mode*. The former produces much faster code, but has limitations that can force Numba to fall back to the latter. To prevent Numba from falling back, and instead raise an error, pass `nopython=True` as we do bellow.

You can read about more arguments that can be passed to `numba.jit()` in the official documentation: http://numba.pydata.org/numba-doc/latest/user/jit.html

In [ ]:
from numba import jit
import numpy as np
import time

x = np.arange(100).reshape(10, 10)

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

## Bubble Sort

Let’s see Numba in action. The following is a Python implementation of bubblesort for NumPy arrays.

In [ ]:
def bubblesort(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

First we’ll create an array of sorted values and randomly shuffle them.

In [ ]:
original = np.arange(0.0, 10.0, 0.01)
original[:10]

In [ ]:
shuffled = original.copy()
np.random.shuffle(shuffled)
shuffled[:10]

Next, create a copy and do a bubble sort on the copy.

In [ ]:
arr_copy = shuffled.copy()
bubblesort(arr_copy)
arr_copy[:10]

In [ ]:
np.array_equal(arr_copy, original)

Now let’s time the execution. Note: we need to copy the array so we sort a random array each time as sorting an already sorted array is faster and so would distort our timing.

In [ ]:
%timeit bubblesort(shuffled.copy())

Ok, so we know how fast the pure Python implementation is. Now we will add the decorator to the function with the paramater nopython set to True:

In [ ]:
fast_bubblesort = jit(nopython=True)(bubblesort)

In [ ]:
fast_bubblesort(shuffled.copy())
%timeit fast_bubblesort(shuffled.copy())

In [ ]:
%timeit np.sort(shuffled.copy())

Using the decorator in this way will defer compilation until the first function execution, so the first execution will be significantly slower.

Numba will infer the argument types at call time, and generate optimized code based on this information. Numba will also be able to compile separate specializations depending on the input types.

## CUDA
### Terminology
Several important terms in the topic of CUDA programming are listed here:

#### host
the CPU
#### device
the GPU
#### host memory
the system main memory
#### device memory
onboard memory on a GPU card
#### kernel
a GPU function launched by the host and executed on the device
#### device function
a GPU function executed on the device which can only be called from the device (i.e. from a kernel or another device function)

In [ ]:
import os
# enabling CUDA simulation:
os.environ['NUMBA_ENABLE_CUDASIM'] = '1'

In [ ]:
from numba import cuda

## Thread positioning

When running a kernel, the kernel function’s code is executed by every thread once. It therefore has to know which thread it is in, in order to know which array element(s) it is responsible for. More complex algorithms may define more complex responsibilities, but the underlying principle is the same.

To help deal with multi-dimensional arrays, CUDA allows you to specify multi-dimensional blocks and grids. In the example above, you could make blockspergrid and threadsperblock tuples of one, two or three integers. Compared to 1-dimensional declarations of equivalent sizes, this doesn’t change anything to the efficiency or behaviour of generated code, but can help you write your algorithms in a more natural way.

One way is for the thread to determines its position in the grid and block and manually compute the corresponding array position:

### Kernel declaration

In [ ]:
@cuda.jit
def my_kernel(io_array):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
    # Block id in a 1D grid
    bx = cuda.blockIdx.x
    # Block width, i.e. number of threads per block
    bw = cuda.blockDim.x
    # Compute flattened index inside the array
    pos = tx + bx * bw
    if pos < io_array.size:  # Check array boundaries
        io_array[pos] *= 2 # do the computation

### Kernel invocation

In [ ]:
import numpy

# Create the data array - usually initialized some other way
data = numpy.ones(256)

# Set the number of threads in a block
threadsperblock = 32 

# Calculate the number of thread blocks in the grid
blockspergrid = (data.size + (threadsperblock - 1)) // threadsperblock

# Now start the kernel
my_kernel[blockspergrid, threadsperblock](data)

# Print the result
print(data)

The following special objects are provided by the CUDA backend for the sole purpose of knowing the geometry of the thread hierarchy and the position of the current thread within that geometry:
* **numba.cuda.threadIdx** - The thread indices in the current thread block. For 1-dimensional blocks, the index (given by the x attribute) is an integer spanning the range from 0 to numba.cuda.blockDim - 1. A similar rule exists for each dimension when more than one dimension is used.
* **numba.cuda.blockDim** - The shape of the block of threads, as declared when instantiating the kernel. This value is the same for all threads in a given kernel, even if they belong to different blocks (i.e. each block is “full”).
* **numba.cuda.blockIdx** - The block indices in the grid of threads launched a kernel. For a 1-dimensional grid, the index (given by the x attribute) is an integer spanning the range from 0 to numba.cuda.gridDim - 1. A similar rule exists for each dimension when more than one dimension is used.
* **numba.cuda.gridDim** - The shape of the grid of blocks, i.e. the total number of blocks launched by this kernel invocation, as declared when instantiating the kernel.

These objects can be 1-, 2- or 3-dimensional, depending on how the kernel was invoked. To access the value at each dimension, use the x, y and z attributes of these objects, respectively.

## Absolute positions

Simple algorithms will tend to always use thread indices in the same way as shown in the example above. Numba provides additional facilities to automate such calculations:
* **numba.cuda.grid(ndim)** - Return the absolute position of the current thread in the entire grid of blocks. ndim should correspond to the number of dimensions declared when instantiating the kernel. If ndim is 1, a single integer is returned. If ndim is 2 or 3, a tuple of the given number of integers is returned.
* **numba.cuda.gridsize(ndim)** - Return the absolute size (or shape) in threads of the entire grid of blocks. ndim has the same meaning as in grid() above.


In [ ]:
import numpy as np
@cuda.jit
def increment_a_2D_array(an_array):
    x, y = cuda.grid(2)
    if x < an_array.shape[0] and y < an_array.shape[1]:
        an_array[x, y] += 1

In [ ]:
import math
an_array = np.ones((64, 64))
threadsperblock = (16, 16)
blockspergrid_x = math.ceil(an_array.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(an_array.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
increment_a_2D_array[blockspergrid, threadsperblock](an_array)
an_array

## Data transfer
Allocate and transfer a numpy ndarray or structured scalar to the device.
* To copy host->device a numpy array:

In [ ]:
ary = np.arange(10)
d_ary = cuda.to_device(ary)

* To copy device->host:

In [ ]:
h_ary = d_ary.copy_to_host()

example:

In [ ]:
import numpy as np
from numba import cuda

arr = np.arange(1000)
d_arr = cuda.to_device(arr)

my_kernel[100, 100](d_arr)

result_array = d_arr.copy_to_host()

## Atomic Operations
Numba provides access to some of the atomic operations supported in CUDA, in the numba.cuda.atomic class.

including: add, compare_and_swap, max, min.

In [ ]:
from numba import cuda
import numpy as np

@cuda.jit
def max_example(result, values):
    """Find the maximum value in values and store in result[0]"""
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x
    i = (bid * bdim) + tid
    cuda.atomic.max(result, 0, values[i])


arr = np.random.rand(16384)
result = np.zeros(1, dtype=np.float64)

max_example[256,64](result, arr)
print(result[0]) # Found using cuda.atomic.max
print(max(arr))  # Print max(arr) for comparision (should be equal!)

## Shared Memory
For maximum performance, a CUDA kernel needs to use shared memory for manual caching of data. CUDA JIT supports the use of cuda.shared.array(shape, dtype) for specifying an NumPy-array-like object inside a kernel.

Note: shared memory is accessed per threadBlock only and is of limited amount (4-16KB hardware dependent)

## cuda syncthreads
Synchronize all threads in the same thread block. This function implements the same pattern as barriers in traditional multi-threaded programming: this function waits until all threads in the block call it, at which point it returns control to all its callers.

Example: calculating the square of sum of array

In [ ]:
from numba import cuda
import numpy as np

@cuda.jit
def sum_example(values,result):
    tx = cuda.threadIdx.x
    partial_sum = 0
    for i in range(tx,values.size,32):
        partial_sum += values[i]
    cuda.atomic.add(result, 0, partial_sum)

    cuda.syncthreads() # Wait until all threads finish
    
    if tx == 0:
        result[0] **= 2

arr = np.ones(32*4)
result = np.zeros(1, dtype=np.int32)

sum_example[1,32](arr, result)
print(result[0]) # Found using cuda.atomic.add
print(sum(arr)**2)  # Print result for comparision (should be equal!)